In [1]:
import numpy as np
import tifffile
from scipy import ndimage
from skimage.filters import gaussian
from skimage.filters import threshold_li
from skimage.measure import label
import os
import glob
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define input and output paths
input_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Projected/1.4Pa-x40/Cadherins/tophat'
output_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/1.4Pa-x40/Cadherins_raw'

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Find all .tif files in the input directory
input_files = glob.glob(os.path.join(input_dir, '*.tif'))
print(f"Found {len(input_files)} .tif files to process")

# Process each file
for input_file in input_files:
    # Get base filename for output
    filename = os.path.basename(input_file)
    base_name = os.path.splitext(filename)[0]

    print(f"Processing: {filename}")

    # Load the image
    img = tifffile.imread(input_file)
    print(f"  Image shape: {img.shape}")

    # Extract Cadherins channel based on image format
    if len(img.shape) == 2:
        # Single channel image (already Cadherins)
        print("  Detected single-channel image, using as is")
        cadherins_channel = img
    elif len(img.shape) == 3 and img.shape[0] == 3:
        # Format is (C, H, W)
        print("  Detected format: (C, H, W)")
        cadherins_channel = img[1]  # Second channel - assuming Cadherins is in channel 1
                                    # Adjust this index if your Cadherins are in a different channel
    elif len(img.shape) == 3 and img.shape[2] == 3:
        # Format is (H, W, C)
        print("  Detected format: (H, W, C)")
        cadherins_channel = img[:, :, 1]  # Second channel - assuming Cadherins is in channel 1
                                          # Adjust this index if your Cadherins are in a different channel
    else:
        print(f"  Unexpected image shape: {img.shape}. Using first channel/plane.")
        if len(img.shape) == 3:
            cadherins_channel = img[0] if img.shape[0] < img.shape[1] else img[:, :, 0]
        else:
            cadherins_channel = img

    # Apply Gaussian blur to reduce noise
    print("  Applying Gaussian blur...")
    cadherins_smoothed = gaussian(cadherins_channel, sigma=1)

    # Apply Li thresholding
    print("  Performing Li thresholding...")
    thresh = threshold_li(cadherins_smoothed)
    binary = cadherins_smoothed > thresh

    # Save the binary mask (main output)
    mask_output = os.path.join(output_dir, f"{base_name}_cadherins_mask.tif")
    tifffile.imwrite(mask_output, binary.astype(np.uint8) * 255)
    print(f"  Saved binary mask to {mask_output}")

    # Print some statistics about the segmentation
    print(f"  Threshold value used: {thresh}")
    print(f"  Processing complete for {filename}\n")

print("All processing complete!")

Mounted at /content/drive
Found 13 .tif files to process
Processing: denoised_1.4Pa_A1_19dec21_40x_L2RA_FlatA_seq001_Cadherins_regional_tophat.tif
  Image shape: (1024, 1024)
  Detected single-channel image, using as is
  Applying Gaussian blur...
  Performing Li thresholding...
  Saved binary mask to /content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/1.4Pa-x40/Cadherins_raw/denoised_1.4Pa_A1_19dec21_40x_L2RA_FlatA_seq001_Cadherins_regional_tophat_cadherins_mask.tif
  Threshold value used: 0.10525799868686057
  Processing complete for denoised_1.4Pa_A1_19dec21_40x_L2RA_FlatA_seq001_Cadherins_regional_tophat.tif

Processing: denoised_1.4Pa_A1_19dec21_40x_L2RA_FlatA_seq002_Cadherins_regional_tophat.tif
  Image shape: (1024, 1024)
  Detected single-channel image, using as is
  Applying Gaussian blur...
  Performing Li thresholding...
  Saved binary mask to /content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/1.4Pa-x40/Cadherins_raw/denoised_1.4Pa_A1_19dec21

In [2]:
import numpy as np
import tifffile
import os
import glob
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define input and output paths
cadherins_input_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/1.4Pa-x40/Cadherins_raw'
golgi_input_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/1.4Pa-x40/Golgi'
cadherins_output_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/1.4Pa-x40/Cadherins'

# Create output directory if it doesn't exist
os.makedirs(cadherins_output_dir, exist_ok=True)

# Find all Cadherins mask files
cadherins_files = glob.glob(os.path.join(cadherins_input_dir, '*.tif'))
print(f"Found {len(cadherins_files)} Cadherins mask files")

# Get all Golgi mask files
golgi_files = glob.glob(os.path.join(golgi_input_dir, '*.tif'))
print(f"Found {len(golgi_files)} Golgi mask files")

# Process each Cadherins mask
for cadherins_file in cadherins_files:
    # Get filename
    cadherins_filename = os.path.basename(cadherins_file)
    print(f"\nProcessing: {cadherins_filename}")

    # Extract common part from filename for matching
    if '_Cadherins_' in cadherins_filename:
        common_part = cadherins_filename.split('_Cadherins_')[0]
    else:
        # Fallback to just removing extension if pattern doesn't match
        common_part = os.path.splitext(cadherins_filename)[0]

    print(f"  Common identifier: {common_part}")

    # Load the Cadherins mask
    cadherins_mask = tifffile.imread(cadherins_file)
    cadherins_binary = cadherins_mask > 0  # Convert to binary boolean mask

    # Look for matching Golgi file
    matching_golgi_file = None

    for golgi_file in golgi_files:
        golgi_filename = os.path.basename(golgi_file)
        # Check if the common part matches and if it's a Golgi file
        if common_part in golgi_filename and '_Golgi_' in golgi_filename:
            matching_golgi_file = golgi_file
            break

    # Extract the original filename without extension for output naming
    original_name_no_ext = os.path.splitext(cadherins_filename)[0]

    # Process and save cleaned mask if a matching file was found
    if matching_golgi_file:
        # Load the Golgi mask
        golgi_mask = tifffile.imread(matching_golgi_file)
        golgi_binary = golgi_mask > 0  # Convert to binary boolean mask

        # Calculate cleaned mask (Cadherins without Golgi overlap)
        cleaned_cadherins = cadherins_binary & ~golgi_binary

        print(f"  Removed Golgi overlap from Cadherins mask using {os.path.basename(matching_golgi_file)}")

        # Calculate statistics
        original_pixels = np.sum(cadherins_binary)
        cleaned_pixels = np.sum(cleaned_cadherins)
        removed_pixels = original_pixels - cleaned_pixels
        removed_percent = (removed_pixels / original_pixels * 100) if original_pixels > 0 else 0
        print(f"  Removed {removed_pixels} pixels ({removed_percent:.2f}% of original)")

        # Save only the cleaned binary mask
        cleaned_file = os.path.join(cadherins_output_dir, f"{original_name_no_ext}_cleaned.tif")
        tifffile.imwrite(cleaned_file, cleaned_cadherins.astype(np.uint8) * 255)
        print(f"  Saved cleaned mask to {cleaned_file}")

    else:
        print(f"  Warning: No matching Golgi mask found for common identifier: {common_part}")

        # Save the original binary mask if no matching Golgi file was found
        original_file = os.path.join(cadherins_output_dir, f"{original_name_no_ext}_original.tif")
        tifffile.imwrite(original_file, cadherins_binary.astype(np.uint8) * 255)
        print(f"  No cleaning performed. Saved original mask to {original_file}")

print("\nAll processing complete!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 13 Cadherins mask files
Found 13 Golgi mask files

Processing: denoised_1.4Pa_A1_19dec21_40x_L2RA_FlatA_seq001_Cadherins_regional_tophat_cadherins_mask.tif
  Common identifier: denoised_1.4Pa_A1_19dec21_40x_L2RA_FlatA_seq001
  Removed Golgi overlap from Cadherins mask using denoised_1.4Pa_A1_19dec21_40x_L2RA_FlatA_seq001_Golgi_regional_tophat_golgi_labels.tif
  Removed 35011 pixels (10.55% of original)
  Saved cleaned mask to /content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/1.4Pa-x40/Cadherins/denoised_1.4Pa_A1_19dec21_40x_L2RA_FlatA_seq001_Cadherins_regional_tophat_cadherins_mask_cleaned.tif

Processing: denoised_1.4Pa_A1_19dec21_40x_L2RA_FlatA_seq002_Cadherins_regional_tophat_cadherins_mask.tif
  Common identifier: denoised_1.4Pa_A1_19dec21_40x_L2RA_FlatA_seq002
  Removed Golgi overlap from Cadherins mask using denoised_1.4Pa_A1_19d